In [1]:
import cv2
from math import inf
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
model_path = '.\\mediapipe_models\\pose_landmarker_heavy.task'

In [3]:
key_name = 'p7s1'
camera_num = 1
file_name = f'c{camera_num}_0090'
frames_per_sec = 25
sample_avi_path = f'.\\gait3d\\Sequences\\{key_name}\\Images\\{file_name}.avi'
frame_size = (540, 960, 3)
print(sample_avi_path)

.\gait3d\Sequences\p7s1\Images\c1_0090.avi


In [4]:
import json
from scripts.parsers import parse_sequences
file_path = 'gait3d\\ListOfSequences.txt'
sequences = parse_sequences(file_path)
print(json.dumps(sequences[key_name], indent=4))

{
    "start_frame": 90,
    "number_of_frames": 120,
    "frame_offset": 0,
    "MoCap_data": true
}


In [5]:
def video_frame_iterator(video_path, frames_n=inf):
    frame_duration_ms = 1000/frames_per_sec
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video.")
        
    iteration = 0
    while True and iteration < frames_n:
        ret, frame = cap.read()
        if not ret:
            break
            
        timestamp = int(frame_duration_ms*iteration)
        iteration += 1
        yield timestamp, frame
        
    cap.release()


In [6]:
for ts, frame in video_frame_iterator(sample_avi_path, 1):
    print(ts)
    print(type(frame))
    print(frame.shape)

0
<class 'numpy.ndarray'>
(540, 960, 3)


In [7]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)


In [8]:
def estimate_poses_for_video(video_path):
    frames_landmarks = []
    with PoseLandmarker.create_from_options(options) as landmarker:
        for frame_ts_ms, frame in video_frame_iterator(video_path):
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_ts_ms)
            frames_landmarks.append(pose_landmarker_result)

    return frames_landmarks


In [9]:
sample_landmarks = estimate_poses_for_video(sample_avi_path)

C:\Users\Miko7\Studia\magisterka\gait-features-identification\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [10]:
sample_landmarks[0].pose_landmarks

[[NormalizedLandmark(x=0.8448679447174072, y=0.2409970611333847, z=-0.07556842267513275, visibility=0.9982926249504089, presence=0.999614953994751),
  NormalizedLandmark(x=0.8467260599136353, y=0.2308843731880188, z=-0.08980179578065872, visibility=0.997923731803894, presence=0.9996803998947144),
  NormalizedLandmark(x=0.8479593396186829, y=0.23035624623298645, z=-0.0899634063243866, visibility=0.9982097148895264, presence=0.9997988343238831),
  NormalizedLandmark(x=0.849114716053009, y=0.22977221012115479, z=-0.0899328738451004, visibility=0.9989669322967529, presence=0.9998242259025574),
  NormalizedLandmark(x=0.8459903001785278, y=0.23061716556549072, z=-0.07058962434530258, visibility=0.9973806738853455, presence=0.9992572665214539),
  NormalizedLandmark(x=0.8466466665267944, y=0.2299623191356659, z=-0.07076970487833023, visibility=0.9974954724311829, presence=0.9993405938148499),
  NormalizedLandmark(x=0.8472632765769958, y=0.2291942834854126, z=-0.07086167484521866, visibility=0.

In [11]:
sample_landmarks[1].pose_world_landmarks

[[Landmark(x=-0.12908835709095, y=-0.5846834778785706, z=-0.232228085398674, visibility=0.9982312321662903, presence=0.9992340803146362),
  Landmark(x=-0.10052894800901413, y=-0.6064781546592712, z=-0.26950162649154663, visibility=0.9978970289230347, presence=0.9994838237762451),
  Landmark(x=-0.10004984587430954, y=-0.6037401556968689, z=-0.2569354176521301, visibility=0.9982000589370728, presence=0.9996994733810425),
  Landmark(x=-0.10015092045068741, y=-0.6065137386322021, z=-0.2605850398540497, visibility=0.998955249786377, presence=0.9997479319572449),
  Landmark(x=-0.09401948004961014, y=-0.6191688179969788, z=-0.24496141076087952, visibility=0.997322678565979, presence=0.9986916184425354),
  Landmark(x=-0.09364727139472961, y=-0.6184149384498596, z=-0.25805068016052246, visibility=0.9974457621574402, presence=0.9988263249397278),
  Landmark(x=-0.08856268972158432, y=-0.602739691734314, z=-0.23649674654006958, visibility=0.9979446530342102, presence=0.9985631108283997),
  Landmar

In [12]:
len(sample_landmarks)

120

In [13]:
mp_pose = mp.solutions.pose

landmarks_num = 33
pose_landmark_names = {i: mp_pose.PoseLandmark(i).name for i in range(landmarks_num)}
connections = list(mp_pose.POSE_CONNECTIONS)

print(pose_landmark_names)
print(connections)

{0: 'NOSE', 1: 'LEFT_EYE_INNER', 2: 'LEFT_EYE', 3: 'LEFT_EYE_OUTER', 4: 'RIGHT_EYE_INNER', 5: 'RIGHT_EYE', 6: 'RIGHT_EYE_OUTER', 7: 'LEFT_EAR', 8: 'RIGHT_EAR', 9: 'MOUTH_LEFT', 10: 'MOUTH_RIGHT', 11: 'LEFT_SHOULDER', 12: 'RIGHT_SHOULDER', 13: 'LEFT_ELBOW', 14: 'RIGHT_ELBOW', 15: 'LEFT_WRIST', 16: 'RIGHT_WRIST', 17: 'LEFT_PINKY', 18: 'RIGHT_PINKY', 19: 'LEFT_INDEX', 20: 'RIGHT_INDEX', 21: 'LEFT_THUMB', 22: 'RIGHT_THUMB', 23: 'LEFT_HIP', 24: 'RIGHT_HIP', 25: 'LEFT_KNEE', 26: 'RIGHT_KNEE', 27: 'LEFT_ANKLE', 28: 'RIGHT_ANKLE', 29: 'LEFT_HEEL', 30: 'RIGHT_HEEL', 31: 'LEFT_FOOT_INDEX', 32: 'RIGHT_FOOT_INDEX'}
[(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)]


In [14]:
def simplify_fames_for_presentation(frames, use_normalized_landmarks=True):
    n = len(frames)
    landmarks_num = 33

    simplified_frames = []

    if use_normalized_landmarks:
        for frame in frames:
            xs = [frame.pose_landmarks[0][i].x for i in range(landmarks_num)]
            ys = [-frame.pose_landmarks[0][i].y for i in range(landmarks_num)]
            zs = [frame.pose_landmarks[0][i].z for i in range(landmarks_num)]
    
            simplified_frames.append((xs, ys, zs))
            
    else: 
        for frame in frames:
            xs = [frame.pose_world_landmarks[0][i].x for i in range(landmarks_num)]
            ys = [-frame.pose_world_landmarks[0][i].y for i in range(landmarks_num)]
            zs = [frame.pose_world_landmarks[0][i].z for i in range(landmarks_num)]
    
            simplified_frames.append((xs, ys, zs))

    return simplified_frames
    

In [15]:
sample_frames_simple = simplify_fames_for_presentation(sample_landmarks)

In [16]:
for i, dim in enumerate(['x', 'y', 'z']):
    print(f"{dim}: {sample_frames_simple[0][i]}")

x: [0.8448679447174072, 0.8467260599136353, 0.8479593396186829, 0.849114716053009, 0.8459903001785278, 0.8466466665267944, 0.8472632765769958, 0.8587758541107178, 0.8556139469146729, 0.8505422472953796, 0.8494491577148438, 0.8935875296592712, 0.8557350635528564, 0.8973052501678467, 0.8501441478729248, 0.8877127170562744, 0.8439985513687134, 0.887445867061615, 0.8410310745239258, 0.8803190588951111, 0.8404601812362671, 0.8793847560882568, 0.8413307666778564, 0.8749555945396423, 0.8515905141830444, 0.859756588935852, 0.84532630443573, 0.852583110332489, 0.8394988775253296, 0.8562908172607422, 0.8444334268569946, 0.8233193159103394, 0.8127117156982422]
y: [-0.2409970611333847, -0.2308843731880188, -0.23035624623298645, -0.22977221012115479, -0.23061716556549072, -0.2299623191356659, -0.2291942834854126, -0.22670824825763702, -0.2280610054731369, -0.2506522536277771, -0.2503798007965088, -0.2854614853858948, -0.28280109167099, -0.37135857343673706, -0.3674367666244507, -0.45211657881736755

In [17]:
sample_frames_simple_not_normalized = simplify_fames_for_presentation(sample_landmarks, False)
for i, dim in enumerate(['x', 'y', 'z']):
    print(f"{dim}: {sample_frames_simple_not_normalized[0][i]}")

x: [-0.15692061185836792, -0.12569093704223633, -0.12493042647838593, -0.12525318562984467, -0.12168724089860916, -0.12140507251024246, -0.11402137577533722, -0.0028038211166858673, -0.006440863013267517, -0.09645050764083862, -0.09553977847099304, 0.11644488573074341, -0.01560862548649311, 0.17244552075862885, -0.02632991038262844, 0.12602777779102325, -0.12073709815740585, 0.11751867085695267, -0.10651946812868118, 0.07016800343990326, -0.10645344108343124, 0.11771393567323685, -0.1193004623055458, 0.04817624017596245, -0.045751556754112244, -0.021540919318795204, -0.07037673145532608, -0.0497204028069973, -0.13453400135040283, -0.07994624972343445, -0.14694863557815552, -0.21399933099746704, -0.2586224377155304]
y: [0.588268518447876, 0.6119582056999207, 0.609531044960022, 0.6120685935020447, 0.6266337633132935, 0.6255468726158142, 0.6104570031166077, 0.6042420268058777, 0.5726945996284485, 0.5775109529495239, 0.5484743118286133, 0.44942206144332886, 0.5214458703994751, 0.2307072579

In [18]:
import plotly.io as pio
import plotly.graph_objects as go
import mediapipe as mp

pio.renderers.default = 'iframe'

mp_pose = mp.solutions.pose
connections = list(mp_pose.POSE_CONNECTIONS)

def show_landmarks_from_mediapipe(frames_landmarks, frames_per_sec=25, frame_size=(540, 960)):
    marker_size = 4
    marker_color = 'blue'
    line_color = 'blue'

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=frames_landmarks[0][0],
        y=frames_landmarks[0][1], 
        mode='markers',
        marker=dict(size=marker_size, color=marker_color),
        showlegend=False
    ))

    for connection in connections:
        fig.add_trace(go.Scatter(
            x=[frames_landmarks[0][0][connection[0]], frames_landmarks[0][0][connection[1]]],
            y=[frames_landmarks[0][1][connection[0]], frames_landmarks[0][1][connection[1]]],
            mode='lines',
            line=dict(color=line_color, width=2),
            showlegend=False
        ))

    frames = []
    for i, curr_frame in enumerate(frames_landmarks):
        frame_data = []

        frame_data.append(go.Scatter(
            x=curr_frame[0],
            y=curr_frame[1],
            mode='markers',
            marker=dict(size=marker_size, color=marker_color)
        ))

        for connection in connections:
            frame_data.append(go.Scatter(
                x=[curr_frame[0][connection[0]], curr_frame[0][connection[1]]],
                y=[curr_frame[1][connection[0]], curr_frame[1][connection[1]]],
                mode='lines',
                line=dict(color=line_color, width=2),
                
            ))

        frames.append(go.Frame(data=frame_data, 
                                name=f'frame {i}',
                                layout=go.Layout(
                                annotations=[dict(
                                    text=f'Frame {i+1}',
                                    x=0.05, y=0.95, xref='paper', yref='paper',
                                    showarrow=False, font=dict(size=16, color="black"))]
        )))

    fig.frames = frames

    fig.update_layout(
        xaxis_title='X',
        yaxis_title='Y',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[-1, 0]),
        title='2D Gait Animation - Mediapipe Processed',
        width=frame_size[1],
        height=frame_size[0],
        annotations=[dict(
            text='Frame 1',
            x=0.05, y=0.95, xref='paper', yref='paper',
            showarrow=False, font=dict(size=16, color="black")
        )],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, dict(frame=dict(duration=1000/frames_per_sec, redraw=True), fromcurrent=True)]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", fromcurrent=True)]
                )
            ]
        )]
    )

    fig.show()

In [19]:
show_landmarks_from_mediapipe(sample_frames_simple)

In [20]:
key_name = 'p7s1'
camera_num = 2 # other camera
file_name = f'c{camera_num}_0090'
frames_per_sec = 25
sample_avi_path = f'.\\gait3d\\Sequences\\{key_name}\\Images\\{file_name}.avi'

sample_landmarks = estimate_poses_for_video(sample_avi_path)
sample_frames_simple = simplify_fames_for_presentation(sample_landmarks)
show_landmarks_from_mediapipe(sample_frames_simple)

C:\Users\Miko7\Studia\magisterka\gait-features-identification\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning:

SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.



In [21]:
import json
from scripts.parsers import parse_sequences

sequences = parse_sequences(file_path)
base_avi_path = '.\\gait3d\\Sequences\\{}\\Images\\{}.avi'
key_name = 'p2s1'
file_name = 'c{}_{}'

sf_str = str(sequences[key_name]["start_frame"])
f_ending = (4 - len(sf_str)) * "0" + sf_str           

avi_paths = [base_avi_path.format(key_name, file_name.format(i, f_ending)) for i in range(1, 5)]
avi_paths

['.\\gait3d\\Sequences\\p2s1\\Images\\c1_0130.avi',
 '.\\gait3d\\Sequences\\p2s1\\Images\\c2_0130.avi',
 '.\\gait3d\\Sequences\\p2s1\\Images\\c3_0130.avi',
 '.\\gait3d\\Sequences\\p2s1\\Images\\c4_0130.avi']

In [22]:
processed_landmarks = []

for camera_avi_path in avi_paths:
    landmarks = estimate_poses_for_video(camera_avi_path)
    processed_landmarks.append(simplify_fames_for_presentation(landmarks))


In [23]:
def adjust_processed_landmarks(processed_landmarks):
    adjusted_processed_landmarks = []

    changes = [(-1, 1, 0), (0, 1, 0), (-1, 0, 0), (0, 0, 0)]
    
    
    for camera in range(4):
        new_camera_frames = []
        for frame in processed_landmarks[camera]:
            new_frame = []
            for i, dim_coords in enumerate(frame):
                new_dim_coords = [item + changes[camera][i] for item in dim_coords]
                new_frame.append(new_dim_coords)
            new_camera_frames.append(new_frame)
        adjusted_processed_landmarks.append(new_camera_frames)

    return adjusted_processed_landmarks
    

In [24]:
import plotly.io as pio
import plotly.graph_objects as go
import mediapipe as mp

pio.renderers.default = 'iframe'

mp_pose = mp.solutions.pose
connections = list(mp_pose.POSE_CONNECTIONS)

def show_landmarks_from_mediapipe_all_cameras(processed_frames_landmarks, seqence_key='', frames_per_sec=25, frame_size=(540, 960)):
    frames_landmarks_all_cameras = adjust_processed_landmarks(processed_frames_landmarks)
    marker_size = 4
    line_width = 2
    colors = ['red', 'green', 'blue', 'orange']

    fig = go.Figure()

    for camera in range(4):
        fig.add_trace(go.Scatter(
            x=frames_landmarks_all_cameras[camera][0][0],
            y=frames_landmarks_all_cameras[camera][0][1],
            mode='markers',
            marker=dict(size=marker_size, color=colors[camera]),
            showlegend=False,
            name=f'c{camera+1} landmarks'
        ))

        for connection in connections:
            fig.add_trace(go.Scatter(
                x=[frames_landmarks_all_cameras[camera][0][0][connection[0]], frames_landmarks_all_cameras[camera][0][0][connection[1]]],
                y=[frames_landmarks_all_cameras[camera][0][1][connection[0]], frames_landmarks_all_cameras[camera][0][1][connection[1]]],
                mode='lines',
                line=dict(color=colors[camera], width=line_width),
                showlegend=False,
                name=f'c{camera+1} bones'
            ))

    num_frames = len(frames_landmarks_all_cameras[0])
    frames = []
    for i in range(num_frames):
        frame_data = []

        for camera in range(4):
            curr_frame = frames_landmarks_all_cameras[camera][i]

            frame_data.append(go.Scatter(
                x=curr_frame[0],
                y=curr_frame[1],
                mode='markers',
                marker=dict(size=marker_size, color=colors[camera]),
                showlegend=False
            ))

            for connection in connections:
                frame_data.append(go.Scatter(
                    x=[curr_frame[0][connection[0]], curr_frame[0][connection[1]]],
                    y=[curr_frame[1][connection[0]], curr_frame[1][connection[1]]],
                    mode='lines',
                    line=dict(color=colors[camera], width=line_width),
                    showlegend=False
                ))

        frames.append(go.Frame(
            data=frame_data,
            name=f'frame {i}',
            layout=go.Layout(
                annotations=[dict(
                    text=f'Frame {i+1}',
                    x=0.5, y=0.5, xref='paper', yref='paper',
                    showarrow=False, font=dict(size=16, color="black")
                )]
            )
        ))

    fig.frames = frames
    camera_desc_positions = [(0.01, 0.99), (0.99, 0.99), (0.01, 0.01), (0.99, 0.01)]

    fig.update_layout(
        xaxis_title='X',
        yaxis_title='Y',
        xaxis=dict(range=[-1, 1]),
        yaxis=dict(range=[-1, 1]),
        title=f'2D mediapipe results from 4 cameras {seqence_key}',
        width=int(frame_size[1] * 1.5),
        height=int(frame_size[0] * 1.5),
        annotations = [
            dict(
                text='Frame 1',
                x=0.5, y=0.5, xref='paper', yref='paper',
                showarrow=False, font=dict(size=16, color="black")
            )
        ] + [
            dict(
                text=f"Camera {i+1}",
                x=camera_desc_positions[i][0], y=camera_desc_positions[i][1],
                xref='paper', yref='paper',
                showarrow=False, font=dict(size=12, color=colors[i])
            ) for i in range(4)
        ],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, dict(frame=dict(duration=1000/frames_per_sec, redraw=True), fromcurrent=True)]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", fromcurrent=True)]
                )
            ]
        )]
    )

    fig.show()


In [25]:
show_landmarks_from_mediapipe_all_cameras(processed_landmarks, key_name)

In [26]:
# PyQT
# OpenPose 3D
# Vicon

In [27]:
len(processed_landmarks)

4

In [28]:
len(processed_landmarks[0])

155

In [29]:
len(processed_landmarks[1])

155

In [30]:
processed_landmarks[0][0]

([0.7655511498451233,
  0.770289421081543,
  0.7722682356834412,
  0.7739561200141907,
  0.768341064453125,
  0.7686821818351746,
  0.768936276435852,
  0.7815371155738831,
  0.7745682597160339,
  0.7703903317451477,
  0.7682839035987854,
  0.8011458516120911,
  0.7750867009162903,
  0.8049696087837219,
  0.779549777507782,
  0.7916091084480286,
  0.771781325340271,
  0.7928096652030945,
  0.7702409625053406,
  0.7864230275154114,
  0.7692295908927917,
  0.7856025099754333,
  0.769853949546814,
  0.7904582619667053,
  0.7707071900367737,
  0.7822328209877014,
  0.7714799046516418,
  0.7803554534912109,
  0.7740541696548462,
  0.7854081988334656,
  0.7771303057670593,
  0.7512503266334534,
  0.7462013363838196],
 [-0.24960125982761383,
  -0.241102397441864,
  -0.24078038334846497,
  -0.24052542448043823,
  -0.24122698605060577,
  -0.2411120980978012,
  -0.24078702926635742,
  -0.24514776468276978,
  -0.24516236782073975,
  -0.2601003348827362,
  -0.2595994770526886,
  -0.300252467393875

In [31]:
pose_landmark_names

{0: 'NOSE',
 1: 'LEFT_EYE_INNER',
 2: 'LEFT_EYE',
 3: 'LEFT_EYE_OUTER',
 4: 'RIGHT_EYE_INNER',
 5: 'RIGHT_EYE',
 6: 'RIGHT_EYE_OUTER',
 7: 'LEFT_EAR',
 8: 'RIGHT_EAR',
 9: 'MOUTH_LEFT',
 10: 'MOUTH_RIGHT',
 11: 'LEFT_SHOULDER',
 12: 'RIGHT_SHOULDER',
 13: 'LEFT_ELBOW',
 14: 'RIGHT_ELBOW',
 15: 'LEFT_WRIST',
 16: 'RIGHT_WRIST',
 17: 'LEFT_PINKY',
 18: 'RIGHT_PINKY',
 19: 'LEFT_INDEX',
 20: 'RIGHT_INDEX',
 21: 'LEFT_THUMB',
 22: 'RIGHT_THUMB',
 23: 'LEFT_HIP',
 24: 'RIGHT_HIP',
 25: 'LEFT_KNEE',
 26: 'RIGHT_KNEE',
 27: 'LEFT_ANKLE',
 28: 'RIGHT_ANKLE',
 29: 'LEFT_HEEL',
 30: 'RIGHT_HEEL',
 31: 'LEFT_FOOT_INDEX',
 32: 'RIGHT_FOOT_INDEX'}

In [32]:
import numpy as np

landmark_i = 0
frame_i = 120
print(pose_landmark_names[landmark_i])

x_cord = 0
y_cord = 0
z_cord = 0

horizontal_coords_cumulated = []
vertical_coords_cumulated = []

for camera_i in range(4):
    horizontal_coords = np.array(processed_landmarks[camera_i][frame_i][0])
    if camera_i > 1:
        horizontal_coords = np.ones(horizontal_coords.shape) - horizontal_coords

    vertical_coords = np.array(processed_landmarks[camera_i][frame_i][1])
    print(horizontal_coords[landmark_i], vertical_coords[landmark_i])

    horizontal_coords_cumulated.append(horizontal_coords)
    vertical_coords_cumulated.append(vertical_coords)
    
new_x = (horizontal_coords_cumulated[1] + horizontal_coords_cumulated[3])/2
new_y = (horizontal_coords_cumulated[0] + horizontal_coords_cumulated[2])/2
new_z = (vertical_coords_cumulated[0] + vertical_coords_cumulated[1] + vertical_coords_cumulated[2] + vertical_coords_cumulated[3])/4

print('3D COORDS:')
print(new_x[0], new_y[0], new_z[0])
        

NOSE
0.061611711978912354 -0.26035618782043457
0.511327862739563 -0.3373311460018158
0.07451176643371582 -0.25774693489074707
0.5003542304039001 -0.27752894163131714
3D COORDS:
0.5058410465717316 0.06806173920631409 -0.28324080258607864


In [33]:
def get_frame_new_coordinates(processed_landmarks, frame_i):
    horizontal_coords_cumulated = []
    vertical_coords_cumulated = []
    
    for camera_i in range(4):
        horizontal_coords = np.array(processed_landmarks[camera_i][frame_i][0])
        if camera_i > 1:
            horizontal_coords = np.ones(horizontal_coords.shape) - horizontal_coords
        vertical_coords = np.array(processed_landmarks[camera_i][frame_i][1])
        
        if np.all(horizontal_coords == 0) or np.all(vertical_coords == 0):
            return (np.zeros(horizontal_coords.shape), np.zeros(horizontal_coords.shape), np.zeros(horizontal_coords.shape))
        
        horizontal_coords_cumulated.append(horizontal_coords)
        vertical_coords_cumulated.append(vertical_coords)
        
    new_x = (horizontal_coords_cumulated[1] + horizontal_coords_cumulated[3])/2
    new_y = (horizontal_coords_cumulated[0] + horizontal_coords_cumulated[2])/2
    new_z = (vertical_coords_cumulated[0] + vertical_coords_cumulated[1] + vertical_coords_cumulated[2] + vertical_coords_cumulated[3])/4

    return (new_x, new_y, new_z)


In [34]:
connections = list(mp_pose.POSE_CONNECTIONS)

print(connections)

[(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)]


In [35]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

x, y, z = get_frame_new_coordinates(processed_landmarks, 0)
labels = list(pose_landmark_names.values())

x_lines = []
y_lines = []
z_lines = []

for i, j in connections:
    x_lines += [x[i], x[j], None]
    y_lines += [y[i], y[j], None]
    z_lines += [z[i], z[j], None]
    
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=3, color='blue'),
            text=labels,
            hoverinfo='text',
            name='Joints'),
        go.Scatter3d(
            x=x_lines, y=y_lines, z=z_lines,
            mode='lines',
            line=dict(color='royalblue', width=2),
            name='Bones'),
        ]
)

fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z',
    xaxis=dict(range=[-1, 1]),
    yaxis=dict(range=[-1, 1]),
    zaxis=dict(range=[-1, 1]),
    aspectmode='cube', 
),
title='3D joints plot from mediapipe approximations',
width=800,
height=800
)

fig.show()

In [36]:
import numpy as np

def calc_angle_from_points(points):
    p1, p2, p3 = points
    vec1 = p1 - p2 
    vec2 = p3 - p2

    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0, 0

    angle_rad = np.arccos(dot_product / (norm_vec1 * norm_vec2))
    angle_deg = np.degrees(angle_rad)

    return angle_deg, angle_rad

def calc_joint_angle(joint_i, connections, x, y, z):
    parents = [parent for parent, child in connections if child == joint_i]
    childs = [child for parent, child in connections if parent == joint_i]
    lever_idx = (parents[0], joint_i, childs[0])
    lever_points = tuple([np.array((x[idx], y[idx], z[idx])) for idx in lever_idx])
    return calc_angle_from_points(lever_points)

calc_joint_angle(26, connections, x, y, z)

(177.92926443969301, 3.105451500124307)

In [37]:
def simplify_fames_for_presentation(frames, use_normalized_landmarks=True):
    n = len(frames)
    landmarks_num = 33

    simplified_frames = []

    if use_normalized_landmarks:
        for frame in frames:
            if len(frame.pose_landmarks) < 1 or len(frame.pose_landmarks[0]) != landmarks_num:
                simplified_frames.append(([0 for _ in range(landmarks_num)], [0 for _ in range(landmarks_num)], [0 for _ in range(landmarks_num)]))
            else:
                xs = [frame.pose_landmarks[0][i].x for i in range(landmarks_num)]
                ys = [-frame.pose_landmarks[0][i].y for i in range(landmarks_num)]
                zs = [frame.pose_landmarks[0][i].z for i in range(landmarks_num)]
                simplified_frames.append((xs, ys, zs))
            
    else: 
        for frame in frames:
            if len(frame.pose_landmarks) < 1 or len(frame.pose_landmarks[0]) != landmarks_num:
                simplified_frames.append(([0 for _ in range(landmarks_num)], [0 for _ in range(landmarks_num)], [0 for _ in range(landmarks_num)]))
            else:
                xs = [frame.pose_world_landmarks[0][i].x for i in range(landmarks_num)]
                ys = [-frame.pose_world_landmarks[0][i].y for i in range(landmarks_num)]
                zs = [frame.pose_world_landmarks[0][i].z for i in range(landmarks_num)]
                simplified_frames.append((xs, ys, zs))

    return simplified_frames
    

In [38]:
from scripts.parsers import parse_sequences
import plotly.graph_objects as go
import numpy as np
import itertools
import json

sequences = parse_sequences(file_path)
base_avi_path = '.\\gait3d\\Sequences\\{}\\Images\\{}.avi'
key_name = 'p7s1'
file_name = 'c{}_{}'

sf_str = str(sequences[key_name]["start_frame"])
f_ending = (4 - len(sf_str)) * "0" + sf_str           

avi_paths = [base_avi_path.format(key_name, file_name.format(i, f_ending)) for i in range(1, 5)]

processed_landmarks = []

for camera_avi_path in avi_paths:
    landmarks = estimate_poses_for_video(camera_avi_path)
    processed_landmarks.append(simplify_fames_for_presentation(landmarks))

frames_per_sec = 25
angle_joints = ['RIGHT_KNEE', 'LEFT_KNEE'] # 'RIGHT_ELBOW', 'LEFT_ELBOW'
angle_joints_i = [key for key, value in pose_landmark_names.items() if value in angle_joints]

x, y, z = get_frame_new_coordinates(processed_landmarks, 0)
labels = list(pose_landmark_names.values())

x_lines = []
y_lines = []
z_lines = []

for i, j in connections:
    x_lines += [x[i], x[j], None]
    y_lines += [y[i], y[j], None]
    z_lines += [z[i], z[j], None]


fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='blue'),
            text=labels,
            hoverinfo='text',
            name='Joints'),
        go.Scatter3d(
            x=x_lines, y=y_lines, z=z_lines,
            mode='lines',
            line=dict(color='royalblue', width=3),
            hoverinfo='none',
            name='Bones'),
        ]
)

frames = []

for f_idx in range(len(processed_landmarks[0])):
    x, y, z = get_frame_new_coordinates(processed_landmarks, f_idx)

    x_lines = []
    y_lines = []
    z_lines = []

    annotation_text = f'Frame {f_idx+1} '
    for joint_i in angle_joints_i:
         annotation_text += f"| {pose_landmark_names[joint_i]}: {calc_joint_angle(joint_i, connections, x, y, z)[0]:.2f}°"
    
    for i, j in connections:
        x_lines += [x[i], x[j], None]
        y_lines += [y[i], y[j], None]
        z_lines += [z[i], z[j], None]

    frames.append(go.Frame(
        data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='blue'),
            text=labels,
            hoverinfo='text',
            name='Joints'),
        go.Scatter3d(
            x=x_lines, y=y_lines, z=z_lines,
            mode='lines',
            line=dict(color='royalblue', width=3),
            hoverinfo='none',
            name='Bones'),
        ],
        name=f'frame {f_idx}',
        layout=go.Layout(
            annotations=[dict(
                text=annotation_text,
                x=0.05, y=0.95, xref='paper', yref='paper',
                showarrow=False, font=dict(size=16, color="black"))]
        )
    ))

fig.frames = frames
    
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[-0.5, 1.5]),
        zaxis=dict(range=[-1, 0]),
        aspectmode='manual',
        aspectratio=dict(x=1, y=2, z=1)
    ),
    title='3D joints plot from mediapipe approximations',
    width=1000,
    height=800,
    annotations=[dict(
        text='Frame 1',
        x=0.05, y=0.95, xref='paper', yref='paper',
        showarrow=False, font=dict(size=16, color="black")
    )],
    updatemenus=[dict(
        type="buttons",
        showactive=True,
        buttons=[
            dict(label="Play",
                 method="animate",
                 args=[None, dict(frame=dict(duration=1000/frames_per_sec, redraw=True), fromcurrent=True)]),

            dict(label="Pause",
                 method="animate",
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")]),
        ]
    )]
)


fig.show()
